# make a new function to make a 2D slice based on 1D Earth model (or 2D or 3D)

In [8]:
# Nobuaki Fuji Oct 2025

# breaking news! confirmed by Lorette and Stéphanie, the CMB is ellipsoidal! 
# when working with geodetic earth models, r_PREM should be corrected as a function of latitude!

using Pkg


cd(@__DIR__)
Pkg.activate("../..")
ParamFile = "../test/testparam.csv"
include("../src/batchRevise.jl") 
myInclude("../src/DSM1D.jl")

  Activating project at `~/Documents/Github/flexibleDSM`
┌ Info: Including with Revise: ../src/DSM1D.jl
└ @ Main /Users/nobuaki/Documents/Github/flexibleDSM/OPTmotors/src/batchRevise.jl:25
ERROR: UndefVarError: `myInclude` not defined in `Main.DSM1D`
Suggestion: check for spelling errors or missing imports.
Stacktrace:
 [1] top-level scope
   @ ~/Documents/Github/flexibleDSM/OPTmotors/src/DSM1D.jl:24
in expression starting at /Users/nobuaki/Documents/Github/flexibleDSM/OPTmotors/src/DSM1D.jl:24


In [4]:
using Geodesy, Interpolations

In [3]:
# Example 1-D model (toy). Replace with your actual model arrays.
# r_model in meters from Earth center (0..R_ref), assume sorted ascending
R_ref = 6371e3                     # model reference radius (m)
r_model = collect(0.0:1000.0:R_ref)  # example radius grid
v_model = 5000 .- 0.5 .* (R_ref .- r_model) ./ 1000  # dummy profile

6372-element Vector{Float64}:
 1814.5
 1815.0
 1815.5
 1816.0
 1816.5
 1817.0
 1817.5
 1818.0
 1818.5
 1819.0
    ⋮
 4996.0
 4996.5
 4997.0
 4997.5
 4998.0
 4998.5
 4999.0
 4999.5
 5000.0

In [5]:
# create an interpolator in radius space (ensure interpolation domain covers used r)
itp = LinearInterpolation(r_model, v_model; extrapolation_bc=Flat())

6372-element extrapolate(interpolate((::Vector{Float64},), ::Vector{Float64}, Gridded(Linear())), Flat()) with element type Float64:
 1814.5
 1815.0
 1815.5
 1816.0
 1816.5
 1817.0
 1817.5
 1818.0
 1818.5
 1819.0
    ⋮
 4996.0
 4996.5
 4997.0
 4997.5
 4998.0
 4998.5
 4999.0
 4999.5
 5000.0

In [6]:
function ellipsoid_surface_radius(lat_deg, lon_deg)
    # Geodesy.jl uses geodetic coordinates:
    g = Geodesy.DEFAULT_GEODESY  # uses WGS84
    gd = Geodesy.LatLon(lat_deg, lon_deg, 0.0) # altitude=0 -> ellipsoid surface
    ecef = Geodesy.cartesian(gd) # returns ECEF Point3
    return norm([ecef.x, ecef.y, ecef.z])  # meters
end

ellipsoid_surface_radius (generic function with 1 method)